<a href="https://colab.research.google.com/github/teukuaditya/Projek-akhir-TBI/blob/main/tbi_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U pip setuptools wheel
!pip -q install -U "jedi>=0.16"
!pip -q install -U "numpy==2.0.2" "pandas==2.2.2" "scikit-learn==1.6.1"
!pip -q install -U faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


## 2) Import & cek versi

In [ ]:
import os, re, glob
import numpy as np
import pandas as pd

import faiss
from sentence_transformers import SentenceTransformer

print("NumPy :", np.__version__)
print("Pandas:", pd.__version__)
print("FAISS :", faiss.__version__)

NumPy : 2.0.2
Pandas: 2.2.2
FAISS : 1.13.2


## Upload dataset



In [ ]:
from google.colab import files
uploaded = files.upload()
list(uploaded.keys())[:10]

Saving datasheet_berita_1.csv to datasheet_berita_1.csv


['datasheet_berita_1.csv']

## 5) Load semua CSV (auto-detect)


In [ ]:
def find_csv_files():
    paths = []
    paths += glob.glob("*.csv")
    paths += glob.glob("dataset/**/*.csv", recursive=True)
    # deduplicate
    paths = sorted(list(dict.fromkeys(paths)))
    return paths

csv_files = find_csv_files()
csv_files

['datasheet_berita.csv', 'datasheet_berita_1.csv']

## 6) Gabungkan dataset dan auto-detect kolom teks berita

Karena nama kolom tiap dataset Kaggle bisa beda, jadi:
- cari kolom yang paling masuk akal untuk **judul** (title/headline/judul)
- cari kolom yang paling masuk akal untuk **isi** (content/text/body/article/description)
- gabungkan jadi kolom full text


In [ ]:
CAND_TITLE = ["title","headline","judul","news_title","article_title","subject","header"]
CAND_TEXT  = ["content","text","body","article","news","description","full_text","isi","berita","paragraph","summary"]

TITLE_COL = None
TEXT_COL  = None

def pick_col(cols, candidates):
    cols_l = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand in cols_l:
            return cols_l[cand]
    return None

def load_and_merge(csv_paths, max_rows=None):
    frames = []
    for p in csv_paths:
        try:
            df = pd.read_csv(p)
        except Exception:
            # fallback encoding/sep guesses
            df = pd.read_csv(p, encoding="latin-1")
        df["_file_"] = os.path.basename(p)
        frames.append(df)
    big = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    if max_rows and len(big) > max_rows:
        big = big.sample(max_rows, random_state=42).reset_index(drop=True)
    return big

raw = load_and_merge(csv_files, max_rows=None)
print("Rows:", len(raw), "Cols:", len(raw.columns))
raw.head(3)

Rows: 4875 Cols: 11


,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source,_file_
0,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...",6 September 2025,https://nasional.kompas.com/read/2025/09/06/14...,"JAKARTA, KOMPAS.com â€“Â Presiden Konfederasi...",Said Iqbal,industri rokok,PT Gudang Garam,PHK massal,phk massal 2025 terbaru,kompas,datasheet_berita.csv
1,"Gempa M 5,3 Guncang Pulau Doi Maluku Utara","Senin, 12 Agu 2024 21:58 WIB",https://news.detik.com/berita/d-7486691/gempa-...,"Gempa bumi berkekuatan magnitudo (M) 5,3 mengg...",pulau doi,gempa,NaN,NaN,NaN,detik,datasheet_berita.csv
2,"Toko Emas Palsu di Riau Dibongkar Polisi, Perh...","Rabu, 30 Jul 2025 22:22 WIB",https://news.detik.com/melindungi-tuah-marwah/...,Satreskrim Polres Bengkalis membongkar praktik...,pemalsuan emas,emas palsu,polres bengkalis,polda riau,melindungi tuah marwah,detik,datasheet_berita.csv


In [ ]:
cols = list(raw.columns)

auto_title = TITLE_COL or pick_col(cols, CAND_TITLE)
auto_text  = TEXT_COL  or pick_col(cols, CAND_TEXT)

print("Auto title col:", auto_title)
print("Auto text  col:", auto_text)

if auto_text is None and auto_title is None:
    raise ValueError("Tidak ketemu kolom title/text. Set TITLE_COL dan TEXT_COL sesuai kolom dataset kamu.")

def clean_text(s):
    return re.sub(r"\s+", " ", str(s) if s is not None else "").strip()

# Optional metadata columns if exist
CAND_DATE = ["date","published_at","publish_date","tanggal","time","created_at"]
CAND_SRC  = ["source","portal","publisher","media","site","news_source","domain","url"]

date_col = pick_col(cols, CAND_DATE)
src_col  = pick_col(cols, CAND_SRC)

raw["_title"] = raw[auto_title].map(clean_text) if auto_title else ""
raw["_text"]  = raw[auto_text].map(clean_text)  if auto_text else ""

raw["_date"]  = raw[date_col].map(clean_text) if date_col else ""
raw["_source"]= raw[src_col].map(clean_text)  if src_col else ""

raw["full_text"] = (raw["_title"] + ". " + raw["_text"]).map(clean_text)
raw = raw[raw["full_text"] != ""].reset_index(drop=True)

raw[["_file_","_source","_date","_title","full_text"]].head(5)

Auto title col: Judul
Auto text  col: Content


,_file_,_source,_date,_title,full_text
0,datasheet_berita.csv,kompas,,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...","Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ..."
1,datasheet_berita.csv,detik,,"Gempa M 5,3 Guncang Pulau Doi Maluku Utara","Gempa M 5,3 Guncang Pulau Doi Maluku Utara. Ge..."
2,datasheet_berita.csv,detik,,"Toko Emas Palsu di Riau Dibongkar Polisi, Perh...","Toko Emas Palsu di Riau Dibongkar Polisi, Perh..."
3,datasheet_berita.csv,detik,,Minyakita Tak Sesuai Ukuran juga Ditemukan di ...,Minyakita Tak Sesuai Ukuran juga Ditemukan di ...
4,datasheet_berita.csv,tempo,,"Pimpin LDP, Sanae Takaichi Calon Kuat PM Perem...","Pimpin LDP, Sanae Takaichi Calon Kuat PM Perem..."


## 7) Chunking berita

Chunking adalah proses memecah teks panjang menjadi potongan potongan kecil


In [ ]:
CHUNK_SIZE = 900
OVERLAP = 160

def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    text = clean_text(text)
    if not text:
        return []
    chunks = []
    start = 0
    while start < len(text):
        end = min(len(text), start + chunk_size)
        ch = text[start:end].strip()
        if ch:
            chunks.append(ch)
        if end == len(text):
            break
        start = max(0, end - overlap)
    return chunks

rows = []
for idx, r in raw.iterrows():
    chunks = chunk_text(r["full_text"])
    for i, ch in enumerate(chunks, start=1):
        rows.append({
            "news_id": idx,
            "chunk_no": i,
            "title": r["_title"],
            "source": r["_source"],
            "date": r["_date"],
            "file": r["_file_"],
            "text": ch
        })

corpus = pd.DataFrame(rows)
print("Chunks:", len(corpus))
corpus.head(5)

Chunks: 17626


,news_id,chunk_no,title,source,date,file,text
0,0,1,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...",kompas,,datasheet_berita.csv,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ..."
1,0,2,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...",kompas,,datasheet_berita.csv,ru mendengar kabar PHK itu dan baru akan menge...
2,0,3,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...",kompas,,datasheet_berita.csv,PT Gudang Garam Tbk (GGRM) belum memberikan pe...
3,0,4,"Viral Isu PHK Buruh Gudang Garam, Said Iqbal: ...",kompas,,datasheet_berita.csv,ih tetap bekerja seperti biasa. Pihaknya juga ...
4,1,1,"Gempa M 5,3 Guncang Pulau Doi Maluku Utara",detik,,datasheet_berita.csv,"Gempa M 5,3 Guncang Pulau Doi Maluku Utara. Ge..."


## 8) Load model embedding (multilingual E5)

Multilingual E5 adalah model AI (deep learning) yang digunakan untuk mengubah teks menjadi vektor angka berdasarkan makna (semantic), dan mendukung banyak bahasa, termasuk Bahasa Indonesia.
Format E5:

-dokumen (passage) = teks berita

-query pencarian = teks pencarian


In [ ]:
model_name = "intfloat/multilingual-e5-small"
model = SentenceTransformer(model_name)

def embed_passages(texts):
    texts = [f"passage: {t}" for t in texts]
    return model.encode(texts, normalize_embeddings=True, show_progress_bar=True)

def embed_query(q):
    return model.encode([f"query: {q}"], normalize_embeddings=True, show_progress_bar=False)[0]

print("Model loaded:", model_name)

Model loaded: intfloat/multilingual-e5-small


## 9) Build embedding + FAISS index

Sistem mengubah teks menjadi embedding vektor dan menyimpannya dalam FAISS IndexFlatIP, di mana karena embedding telah dinormalisasi, perhitungan inner product dapat langsung digunakan sebagai ukuran kemiripan makna (cosine similarity) antara query dan berita..


In [ ]:
texts = corpus["text"].tolist()
emb = embed_passages(texts).astype("float32")
print("Embeddings:", emb.shape)

d = emb.shape[1]
index = faiss.IndexFlatIP(d)
index.add(emb)
print("FAISS index size:", index.ntotal)

Batches:   0%|          | 0/551 [00:00<?, ?it/s]

Embeddings: (17626, 384)
FAISS index size: 17626


## 10) Hybrid logic: semantic + validasi keyword saat skor ragu

Hybrid logic digunakan untuk menentukan hasil pencarian berdasarkan tingkat kepercayaan skor semantic. Jika skor kemiripan tinggi (≥ HIGH), hasil langsung diterima. Jika skor berada pada rentang menengah (LOW–HIGH), hasil hanya diterima apabila lolos validasi keyword. Sementara itu, hasil dengan skor rendah (< LOW) langsung dibuang karena dianggap tidak relevan.



In [ ]:
HIGH = 0.50
LOW  = 0.15

def tokenize(q):
    toks = re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9_]+", (q or "").lower())
    return [t for t in toks if len(t) >= 3]

def simple_synonyms(tokens):
    # perluasan sederhana ejaan umum
    expanded = set(tokens)
    for t in tokens:
        if t == "analisa": expanded.add("analisis")
        if t == "analisis": expanded.add("analisa")
        if t == "cabe": expanded.add("cabai")
        if t == "cabai": expanded.add("cabe")
    return expanded

def keyword_validate(text, query):
    toks = tokenize(query)
    if not toks:
        return True
    expanded = simple_synonyms(toks)
    t = text.lower()
    return any(k in t for k in expanded)

def highlight(text, query):
    toks = sorted(set(tokenize(query)), key=len, reverse=True)[:12]
    out = text
    for t in toks:
        out = re.sub(rf"\b({re.escape(t)})\b", r"[[\1]]", out, flags=re.IGNORECASE)
    return out

def search(query, topk=10):
    q = clean_text(query)
    if not q:
        return pd.DataFrame()

    qv = embed_query(q).astype("float32").reshape(1, -1)
    scores, ids = index.search(qv, topk*5)
    scores = scores.flatten()
    ids = ids.flatten()

    results = []
    for s, i in zip(scores, ids):
        if i < 0:
            continue
        row = corpus.iloc[int(i)]
        accept = False
        if s >= HIGH:
            accept = True
        elif s >= LOW:
            accept = keyword_validate(row["text"], q)
        else:
            accept = False

        if accept:
            results.append({
                "rank": len(results) + 1,
                "score": float(s),
                "title": row["title"],
                "source": row["source"],
                "date": row["date"],
                "file": row["file"],
                "chunk_no": int(row["chunk_no"]),
                "snippet": highlight(row["text"][:420], q)
            })
        if len(results) >= topk:
            break

    return pd.DataFrame(results)

# Demo
search("kenaikan harga beras 2025", topk=8)

,rank,score,title,source,date,file,chunk_no,snippet
0,1,0.894688,Bagaimana Bapanas Redam Harga Beras di 214 Kab...,tempo,,datasheet_berita.csv,2,rapkan memberi dampak nyata dalam menekan [[ha...
1,2,0.892394,"Menteri Desa PDT Ingin SDM Desa Dioptimalkan, ...",kompas,,datasheet_berita.csv,4,"""Jadi, [[2025]] tidak impor [[beras]] untuk ko..."
2,3,0.889466,Prediksi Harga Emas Sepekan ke Depan,tempo,,datasheet_berita.csv,2,"25 basis poin ke level 4-4,25 persen.â€œProyek..."
3,4,0.886424,Jalan Pintas Anggaran Swasembada Pangan,tempo,,datasheet_berita.csv,2,t menjadikan Indonesia tidak hanya mencapai sw...
4,5,0.882772,Ombudsman: Kebijakan Perberasan Nasional Belum...,tempo,,datasheet_berita.csv,2,"iklan, klik di sini Yeka menyinggung catatan B..."
5,6,0.880393,Ombudsman: Kebijakan Perberasan Nasional Belum...,tempo,,datasheet_berita.csv,1,Ombudsman: Kebijakan Perberasan Nasional Belum...
6,7,0.878763,Menteri Amran Bantah Beras Mahal Akibat Bulog ...,tempo,,datasheet_berita.csv,2,mayoritasnya dibeli oleh perusahaan swasta. â€...
7,8,0.877099,Bagaimana Bapanas Redam Harga Beras di 214 Kab...,tempo,,datasheet_berita.csv,5,akukan pemerintah berjalan efektif dan bermanf...


## 11) Simpan index & corpus (biar bisa dipakai ulang)

File yang disimpan:
news_faiss.index
news_corpus_chunks.csv


In [ ]:
faiss.write_index(index, "news_faiss.index")
corpus.to_csv("news_corpus_chunks.csv", index=False)
print("OK: news_faiss.index dan news_corpus_chunks.csv tersimpan")

OK: news_faiss.index dan news_corpus_chunks.csv tersimpan


## 12) query uji + export hasil

In [ ]:
queries = [
    "banjir"
]

all_out = []
for q in queries:
    r = search(q, topk=8)
    r["query"] = q
    all_out.append(r)

out = pd.concat(all_out, ignore_index=True) if all_out else pd.DataFrame()
out.to_csv("news_search_results.csv", index=False)
print("OK: news_search_results.csv tersimpan")
out.head(12)

OK: news_search_results.csv tersimpan


,rank,score,title,source,date,file,chunk_no,snippet,query
0,1,0.854025,"Bali Banjir Parah, Ini Data Hujan yang Terjadi...",tempo,,datasheet_berita.csv,4,"npasar. Per Rabu tengah hari, 10 September 202...",banjir
1,2,0.852288,"20 Titik di Tangsel Banjir Malam Ini, Ada yang...",detik,,datasheet_berita.csv,1,"20 Titik di Tangsel [[Banjir]] Malam Ini, Ada ...",banjir
2,3,0.851441,BPBD: Banjir Rob Jakarta akibat Penurunan Tana...,kompas,,datasheet_berita.csv,4,"Angke, Penjaringan. Di wilayah ini, [[banjir]]...",banjir
3,4,0.850113,BNPB Catat 37.058 KK di Jabodetabek Terdampak ...,detik,,datasheet_berita.csv,7,n dengan sebanyak 1.373 KK atau 4.157 jiwa ter...,banjir
4,5,0.848413,Kemensos Salurkan Bantuan Korban Banjir Bandan...,tempo,,datasheet_berita.csv,1,Kemensos Salurkan Bantuan Korban [[Banjir]] Ba...,banjir
5,6,0.847929,Banjir Bandang Terjang Permukiman Adat di Didi...,detik,,datasheet_berita.csv,1,[[Banjir]] Bandang Terjang Permukiman Adat di ...,banjir
6,7,0.846709,BNPB Catat 37.058 KK di Jabodetabek Terdampak ...,detik,,datasheet_berita.csv,6,tigasi dan pencegahan terhadap potensi bahaya ...,banjir
7,8,0.844829,Kondisi Banjir Kelapa Gading Seharian Ini: Mal...,detik,,datasheet_berita.csv,1,Kondisi [[Banjir]] Kelapa Gading Seharian Ini:...,banjir
